In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import numpy as np

from models.autoencoder import Autoencoder
from dataset_extracted import ExtractedFeatureDatasetOnlyFeature
import os
import time

In [2]:
optimizer = optim.Adam([], lr=0.001)
optimizer.load_

AttributeError: module 'torch.optim' has no attribute 'A'

In [2]:
latent = 512
lr = 0.0001
BATCH = 1
EPOCH = 100

In [3]:
feature_path = '../MVAD/I3D_rgb_kinetics/train'
with open('../MVAD/train_fine') as f:
    files = f.readlines()
    feature_files = list(map(lambda file: os.path.join(feature_path, str.strip(file) + '.npy'), files))
    feature_files.extend(list(map(lambda file: os.path.join(feature_path + '_fliped', str.strip(file) + '.npy'), files)))
dataset = ExtractedFeatureDatasetOnlyFeature(None ,feature_files=feature_files)
dataloader = DataLoader(dataset, BATCH, True)

In [4]:
device = torch.device('cuda:1')

In [5]:
model = Autoencoder(512)
model.to(device)
model.train()

Autoencoder(
  (encoder_linear1): Linear(in_features=1024, out_features=512, bias=True)
  (decoder_linear1): Linear(in_features=512, out_features=1024, bias=True)
)

In [6]:
optimizer = optim.Adam(model.parameters(), lr)
criterion = nn.L1Loss()

In [ ]:
start_time = time.time()
for epoch in range(1, EPOCH + 1):
    train_loss = 0
    train_accuracy = 0
    for batch, feature in enumerate(dataloader, start=1):
        feature = feature.to(device)
        out = model(feature)
        loss = criterion(out, feature)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        accuracy = 0
        train_loss += loss
        train_accuracy += accuracy
        
    print('epoch {} {:.1f}s - train_loss: {:.6f} train_accuracy: {:.4f}'.format(epoch, time.time() - start_time, train_loss / batch, train_accuracy / batch))
    if epoch % 10:
        torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),}, '../checkpoint/autoencoder{}'.format(epoch // 10))

epoch 1 158.6s - train_loss: 0.034122 train_accuracy: 0.0000
epoch 2 312.4s - train_loss: 0.027150 train_accuracy: 0.0000
epoch 3 462.3s - train_loss: 0.026602 train_accuracy: 0.0000
epoch 4 612.7s - train_loss: 0.026309 train_accuracy: 0.0000
epoch 5 766.0s - train_loss: 0.026122 train_accuracy: 0.0000
